In [1]:
# imports
import os
import pickle
import time
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# changing directory
# Run this cell only once!!
path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)

In [3]:
# get the data

# general information
continents = pd.read_csv('data/continents.csv')
countries = pd.read_csv("data/countries.csv")

# timeseries
global_ts = pd.read_csv("data/global_ts.csv")
country_confirmed = pd.read_csv("data/country_confirmed.csv")
country_deaths = pd.read_csv("data/country_deaths.csv")
country_recovered = pd.read_csv("data/country_recovered.csv")
vac_global = pd.read_csv("data/vac_global.csv")
vac_country = pd.read_csv("data/vac_country.csv")

# pickle file has only data for text and nothing for chart or analysis
with open("data/global_update.pkl", "rb") as file:
    global_update = pickle.load(file)


In [4]:
# updating index for all dataframes
continents.index = continents['continent']
continents.drop(columns=['continent'], inplace=True)
country_confirmed.index = country_confirmed['country']
country_confirmed.drop(columns=['country'], inplace=True)
country_deaths.index = country_deaths['country']
country_deaths.drop(columns=['country'], inplace=True)
country_recovered.index = country_recovered['country']
country_recovered.drop(columns=['country'], inplace=True)
vac_country.index = vac_country['country']
vac_country.drop(columns=['country'], inplace=True)
global_ts.index = global_ts['category']
global_ts.drop(columns=['category'], inplace=True)
# vac_global is a single time series so no need to change its index

In [5]:
def combined_country(country,confirmed_df, recovered_df, deaths_df, vac_df):
    comb_df = pd.DataFrame()
    comb_df["confirmed"] = confirmed_df.loc[country]
    comb_df["deaths"] = deaths_df.loc[country]
    comb_df["recovered"] = recovered_df.loc[country]
    comb_df["vaccined"] = vac_df.loc[country]
    return comb_df


In [6]:
# Example
combined_country("India", country_confirmed, country_recovered, country_deaths, vac_country)

,confirmed,deaths,recovered,vaccined
1/22/20,0,0,0,NaN
1/23/20,0,0,0,NaN
1/24/20,0,0,0,NaN
1/25/20,0,0,0,NaN
1/26/20,0,0,0,NaN
...,...,...,...,...
9/1/21,32857937,439529,0,661576916.0
9/2/21,32903289,439895,0,669775627.0
9/3/21,32945907,440225,0,675654935.0
9/4/21,32988673,440533,0,677744969.0


In [7]:
def combined_global(cases_df, vac_df):
    comb_df = pd.DataFrame()
    comb_df["confirmed"] = cases_df.loc["cases"]
    comb_df["deaths"] = cases_df.loc["deaths"]
    comb_df["recovered"] = cases_df.loc["recovered"]
    comb_df["vaccined"] = vac_df.loc[0]
    return comb_df

In [8]:
# Example
combined_global(global_ts, vac_global)

,confirmed,deaths,recovered,vaccined
1/22/20,557,17,30,NaN
1/23/20,655,18,32,NaN
1/24/20,941,26,39,NaN
1/25/20,1433,42,42,NaN
1/26/20,2118,56,56,NaN
...,...,...,...,...
9/1/21,218431264,4531889,0,5.387017e+09
9/2/21,219062869,4541463,0,5.418303e+09
9/3/21,219821446,4553701,0,5.442297e+09
9/4/21,220282354,4561159,0,5.460115e+09


In [9]:
# single function building off last two
def combined_timeseries(location="global",*dfs):
    # remember to put the correct order of dfs in each case
    if location == "global":
        return combined_global(*dfs)
    return combined_country(location,*dfs)

In [10]:
# Example 1
combined_timeseries("UK", country_confirmed, country_recovered, country_deaths, vac_country)

,confirmed,deaths,recovered,vaccined
1/22/20,0,0,0,NaN
1/23/20,0,0,0,NaN
1/24/20,0,0,0,NaN
1/25/20,0,0,0,NaN
1/26/20,0,0,0,NaN
...,...,...,...,...
9/1/21,6856933,133066,0,91155368.0
9/2/21,6894915,133244,0,91314745.0
9/3/21,6937270,133365,0,91456622.0
9/4/21,6973995,133485,0,91456622.0


In [30]:
# Example 2
df = combined_timeseries("global", global_ts, vac_global)

In [28]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df["confirmed"], name="confirmed"))
fig.add_trace(go.Scatter(x=df.index, y=df["deaths"], name="deaths"))
fig.add_trace(go.Scatter(x=df.index, y=df["recovered"], name="recovered"))
fig.add_trace(go.Scatter(x=df.index, y=df["vaccined"], name="vaccined"))
fig.update_traces(hoverinfo='text+name', mode='lines')
fig.update_layout(legend=dict(font_size=16))
fig.show()


In [40]:
def get_plot(time_series, name):
    color = (
        "#f54842" if "deaths" in name else "#45a2ff" if "cases" in name else "#42f587"
    )
    fig = px.bar(
        time_series,
        x=time_series.index,
        y="confirmed",
        color_discrete_sequence=[color] * len(time_series),
        template='plotly_dark',
    )
    return fig

In [41]:
get_plot(df["confirmed"][-30:],"cases")